# Segmenting and Clustering Neighborhoods in Toronto

## Scraping data

In [2]:
from bs4 import BeautifulSoup
import requests
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(response)
soup = BeautifulSoup(response.content, 'lxml')
#print(soup.prettify())

<Response [200]>


In [44]:
import pandas as pd

column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

for tr in soup.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Data cleaning

In [47]:
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### If a cell has a borough but a Not assigned neighborhood

In [52]:
# The neighborhood will be the same as the borough
indexes = df[ df['Borough'] =='Not assigned'].index
df.drop(indexes , inplace=True)
df.head(15)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### Combining into one row with the neighborhoods separated with a comma

In [53]:
merged = df.groupby(['Postalcode', 'Borough'], sort = False).agg(', '.join)
final_df = merged.reset_index()
final_df.head(20)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [56]:
final_df.shape

(103, 3)